In [ ]:
import json
import time
from itertools import permutations, combinations

# ------------------------------
# Hilfsfunktionen
# ------------------------------
def is_valid_truth_booth(pairing, truth_booths):
    for tb in truth_booths:
        man, woman, is_match = tb["man"], tb["woman"], tb["is_match"]
        if (man in pairing and pairing[man] == woman) != is_match:
            return False
    return True

def respects_perfect_matches(pairing, truth_booths):
    for tb in truth_booths:
        if tb["is_match"]:
            if pairing.get(tb["man"]) != tb["woman"]:
                return False
    return True

def match_score(pairing, ceremony):
    expected_pairs = {(p["man"], p["woman"]) for p in ceremony["pairs"]}
    no_pair = ceremony.get("no_pair", None)
    return sum(
        1 for man, woman in pairing.items()
        if (man, woman) in expected_pairs and woman != no_pair
    )

def is_valid_ceremonies(pairing, ceremonies):
    for ceremony in ceremonies:
        expected_score = ceremony["score"]
        if match_score(pairing, ceremony) != expected_score:
            return False
    return True

def evaluate_pairing(pairing, truth_booths, ceremonies):
    score = 0
    # Add points for each truth booth that is respected
    for tb in truth_booths:
        man, woman, is_match = tb["man"], tb["woman"], tb["is_match"]
        if (man in pairing and pairing[man] == woman) == is_match:
            score += 1
    # Add points for each ceremony score that matches
    for ceremony in ceremonies:
        if match_score(pairing, ceremony) == ceremony["score"]:
            score += 1
    return score

# ------------------------------
# Hauptfunktion: Suche der gültigen Lösung
# ------------------------------

def find_unique_solution(men, women, truth_booths, all_ceremonies, start=5, patience=1000000):
    # 1. Small/large‑Gruppen bestimmen
    if len(men) <= len(women):
        small, large, small_is_men = men, women, True
    else:
        small, large, small_is_men = women, men, False

    min_size = len(small)

    # 2. Nacheinander immer eine Ceremony mehr nutzen
    for n in range(start, len(all_ceremonies) + 1):
        current = all_ceremonies[:n]
        valid = []
        no_improve = 0
        best_score = -1

        print(f"Teste mit den ersten {n} Matching Nights…")

        for subset in combinations(large, min_size):
            for perm in permutations(subset):
                pairing = dict(zip(small, perm)) if small_is_men \
                          else {man: woman for man, woman in zip(perm, small)}

                if not (is_valid_truth_booth(pairing, truth_booths)
                        and respects_perfect_matches(pairing, truth_booths)):
                    continue

                if is_valid_ceremonies(pairing, current):
                    valid.append(pairing)
                    if len(valid) > 1:
                        break
                else:
                    score = evaluate_pairing(pairing, truth_booths, current)
                    if score > best_score:
                        best_score, no_improve = score, 0
                    else:
                        no_improve += 1
                    if no_improve >= patience:
                        break
            if len(valid) > 1 or no_improve >= patience:
                break

        if len(valid) == 1:
            print(f"✅ Einzig gültige Lösung bei n={n} gefunden.")
            return n, valid[0]
        else:
            print(f"→ {len(valid)} Lösungen bei n={n}. Füge nächste Ceremony hinzu.")

    print("❌ Selbst mit allen Matching Nights keine eindeutige Lösung.")
    return None, None

# ------------------------------
# Ausführung
# ------------------------------
if __name__ == "__main__":
    # Pfad zur kodierten JSON
    path = ".\\src\\ger\\normal\\season_test6.json"
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 1) Teilnehmer‑Codes extrahieren
    men   = [m["code"] for m in data["participants"]["men"]]
    women = [w["code"] for w in data["participants"]["women"]]

    # 2) Mapping für die Ausgabe vorbereiten
    code_to_name = {
        **{m["code"]: m["name"] for m in data["participants"]["men"]},
        **{w["code"]: w["name"] for w in data["participants"]["women"]},
    }

    truth_booths   = data["truth_booths"]
    ceremonies     = data["match_ceremonies"]

    # 3) Suche starten
    start_time = time.time()
    n_unique, unique_solution = find_unique_solution(
        men, women, truth_booths, ceremonies, start=3
    )
    elapsed = time.time() - start_time

    # 4) Ergebnis ausgeben
    if n_unique:
        print(f"\nDu brauchst mindestens {n_unique} Matching Nights für eine eindeutige Lösung:")
        for man, woman in unique_solution.items():
            print(f"  {code_to_name[man]} ({man}) ↔ {code_to_name[woman]} ({woman})")
    else:
        print("\nKeine eindeutige Lösung gefunden.")
    print(f"Laufzeit: {elapsed:.2f} s")

Teste mit den ersten 3 Matching Nights…
→ 2 Lösungen bei n=3. Füge nächste Ceremony hinzu.
Teste mit den ersten 4 Matching Nights…
→ 2 Lösungen bei n=4. Füge nächste Ceremony hinzu.
Teste mit den ersten 5 Matching Nights…
✅ Einzig gültige Lösung bei n=5 gefunden.

Du brauchst mindestens 5 Matching Nights für eine eindeutige Lösung:
  Asaf (M1) ↔ Kaylen (W6)
  Cam (M2) ↔ Julia (W5)
  Cameron (M3) ↔ Mikala (W7)
  Giovanni (M4) ↔ Francesca (W4)
  John (M5) ↔ Victoria (W10)
  Morgan (M6) ↔ Tori (W9)
  Prosper (M7) ↔ Emma (W3)
  Sam (M8) ↔ Alyssa (W1)
  Stephen (M9) ↔ Nicole (W8)
  Tyler (M10) ↔ Camille (W2)
Laufzeit: 6.86 s
